# typedLang
cbvLang 노트북에서 정의했던 내용을 가져와서 타입 검사가 용이하도록
Expr 문법을 조금 수정하였다.

$\displaystyle
\tau ::= \mathtt{unit} \mid \mathtt{bool} \mid \mathtt{int} \mid \tau \times \tau \mid \tau + \tau \mid \tau\to\tau$

In [1]:
-- (Int + Int)
data Wepon = GUN Int | SWORD Int

In [2]:
-- simple types for Expr
data Ty
  = UNIT -- 값이 딱 하나 존재
  | BOOL -- 값이 두 개 존재
  | INT  -- 값이 무한히 많이 존재
  | Ty :* Ty   -- product type (pair) 순서쌍 타입 -- 구조체
  | Ty :+ Ty   -- sum type (tagged union) -- 유니온 태그 있는
  | Ty :-> Ty  -- 함수 타입
  deriving (Show, Eq)

infixr 9 :*
infixr 8 :+
infixr 7 :->

In [3]:
-- 변수 이름은 문자열로 나타낸다
type Nm = String

-- 장난감 함수형 언어 = 람다식 + ...
data Expr
  = Var Nm                 -- x, y, z, ...
  | Unit                   -- ()
  | B Bool                 -- False, True
  | I Int                  -- 0, -1, 1, -2, 2, -3, 3, ...
  | Pair Expr Expr         -- (e1,e2)
  | Fst Expr               -- fst e
  | Snd Expr               -- snd e
  | InL Expr               -- InL e
  | InR Expr               -- InR e
  | Case Expr Expr Expr    -- 하스켈이라면 대략 다음에 해당 case e of { inL x1 -> e1 x1; inR x2 -> e2 x2 }
  | Lam (Nm,Ty) Expr       -- \x:t.e
  | Rec Nm Expr            -- rec f e
  | App Expr Expr          -- e1 e2
  | If Expr Expr Expr      -- if e then e1 else e2
  | Let (Nm,Ty) Expr Expr  -- let x:t=e2 in e
  | Add Expr Expr          -- e1 + e2
  | Mul Expr Expr          -- e1 * e2
  | Neg Expr               -- - e
  | Eq Expr Expr           -- e1 == e2
  | Ne Expr Expr           -- e1 /= e2
  | Lt Expr Expr           -- e1 <  e2
  | Gt Expr Expr           -- e1 >  e2
  | Le Expr Expr           -- e1 <= e2
  | Ge Expr Expr           -- e1 >= e2
  | And Expr Expr          -- e1 && e2
  | Or Expr Expr           -- e1 || e2
  | Not Expr               -- not e
  deriving (Show, Eq)

In [4]:
import IHaskell.Display

dispTy t = Display [html("<code>"++dpTy t++"</code>")]
dispExpr e = Display [html("<code>"++dpExpr e++"</code>")]


dpTy UNIT = "unit"
dpTy UNIT = "bool"
dpTy INT = "int"
dpTy (t1 :-> t2) = dpTyArrL t1++"->"++dpTy t2
dpTy (t1 :+ t2) = dpTySum t1++"+"++dpTySum t2
dpTy (t1 :* t2) = dpTyProd t1++"*"++dpTyProd t2

dpTyArrL t@(_ :-> _) = paren $ dpTy t
dpTyArrL t = dpTy t

dpTySum t@(_ :-> _) = paren $ dpTyProd t
dpTySum t@(_ :+ _) = paren $ dpTyProd t
dpTySum t = dpTy t

dpTyProd t@(_ :-> _) = paren $ dpTy t
dpTyProd t@(_ :+ _) = paren $ dpTy t
dpTyProd t@(_ :* _) = paren $ dpTy t
dpTyProd t = dpTy t

dpExpr (Var x) = x
dpExpr (Lam (x,t) e) = "λ" ++ x ++":"++dpTy t++ "." ++ dpExpr e
dpExpr (App e1 e2) = dpt e1 ++ " " ++ dps e2
dpExpr (I n) = show n
dpExpr (B b) = show b
dpExpr Unit = "()"
dpExpr (Pair e1 e2) = "("++dpExpr e1++","++dpExpr e2++")"
dpExpr (Fst e) = "fst" ++ dpe e
dpExpr (Snd e) = "snd" ++ dpe e
dpExpr (InL e) = "inL" ++ dpe e
dpExpr (InR e) = "inR" ++ dpe e
dpExpr (Case e e1 e2) = "case " ++ dpe e ++ " of { inL: "++dpExpr e1++"; inR: "++dpExpr e2++" }"
dpExpr (Add e1 e2) = dpe e1++" + "++dpe e2
dpExpr (Mul e1 e2) = dpe e1++" * "++dpe e2
dpExpr (Neg e) = "-"++dpe e
dpExpr (Eq e1 e2) = dpe e1++" == "++dpe e2
dpExpr (Ne e1 e2) = dpe e1++" /= "++dpe e2
dpExpr (Lt e1 e2) = dpe e1++" < "++dpe e2
dpExpr (Gt e1 e2) = dpe e1++" > "++dpe e2
dpExpr (Le e1 e2) = dpe e1++" <= "++dpe e2
dpExpr (Ge e1 e2) = dpe e1++" >= "++dpe e2
dpExpr (And e1 e2) = dpe e1++" && "++dpe e2
dpExpr (Or e1 e2) = dpe e1++" || "++dpe e2
dpExpr (Not e) = "not "++ dpe e
dpExpr (Let (x,t) e2 e) = "let "++x++":"++dpTy t++" = "++dpt e2++" in "++dpExpr e
dpExpr (Rec f e) = "rec "++f++" "++dpExpr e
dpExpr (If e e1 e2) = "if "++dpe e++" then "++dpe e1++" else "++dpe e2

dpt t@(Lam{}) = paren (dpExpr t)
dpt t@(Let{}) = paren (dpExpr t)
dpt t@(If{})  = paren (dpExpr t)
dpt t@(Rec{}) = paren (dpExpr t)
dpt t         = dpExpr t

dps s@(I _)   = dpExpr s
dps s@(B _)   = dpExpr s
dps s@Unit    = dpExpr s
dps s@(Pair{})    = dpExpr s
dps s@(Var{}) = dpExpr s
dps s         = paren (dpExpr s)

dpe = dps


paren s = "(" ++ s ++ ")"

In [5]:
INT :-> INT :-> INT == INT :-> (INT :-> INT)
INT :-> INT :-> INT == (INT :-> INT) :-> INT
INT :* INT :+ INT :* INT == (INT :* INT) :+ (INT :* INT)
INT :-> INT :+ INT :-> INT == INT :-> (INT :+ INT) :-> INT
INT :-> INT :* INT :-> INT == INT :-> (INT :* INT) :-> INT

True

False

True

True

True

In [6]:
dpTy (INT :-> INT :-> INT)
dpTy (INT :-> (INT :-> INT))
dpTy ((INT :-> INT) :-> INT)
dpTy (INT :-> INT :-> INT)
dpTy $ (INT :-> INT) :-> INT
dpTy $ INT :* INT :+ INT :* INT
dpTy $ INT :* (INT :+ INT) :* INT
dpTy $ INT :-> INT :+ INT :-> INT
dpTy $ INT :-> (INT :+ INT) :-> INT
dpTy $ INT :-> INT :* INT :-> INT
dpTy $ (INT :-> INT) :* (INT :-> INT)

dpTy $ INT :* INT :* INT :* INT
dpTy $ INT :+ INT :+ INT :+ INT

"int->int->int"

"int->int->int"

"(int->int)->int"

"int->int->int"

"(int->int)->int"

...

"int->int+int->int"

"int->int*int->int"

"(int->int)*(int->int)"

"int*(int*(int*int))"

"int+((int+((int+int))))"

In [7]:
idExpr = Lam ("x",INT) (Var "x")

putStrLn $ dpExpr $ Lam ("y",INT) (App idExpr (App idExpr (Var "y")))

dispExpr $ Lam ("y",INT) (App idExpr (App idExpr (Var "y")))

λy:int.(λx:int.x) ((λx:int.x) y)

λy:int.(λx:int.x) ((λx:int.x) y)

In [8]:
putStrLn $ dpExpr $ I 3 `Add` I 4
putStrLn $ dpExpr $ Let ("id",INT) idExpr (App (Var "id") (I 5))

3 + 4

let id:int = (λx:int.x) in id 5

# 장난감 함수형 언어의 call-by-value evaluation
call-by-value evaluation (줄여서 CVB evaluation)은
적극적 계산법(eager evaluation)이라고 부르기도 한다.

TODO cbvLang에 없었던 요소들에 대한 규칙들 중요한 것들을 추려서 제시하였다. TODO

$\displaystyle\begin{array}{rcrcl}
\sigma&\in&\textit{Env} &=&
\textit{Nm} \longrightarrow^{\hspace{-2.7ex}\textrm{fin}} \textit{Val} \\
v&\in&\textit{Val} &=& \textit{Unit}\;\cup\;\textit{Bool}\;\cup\;\textit{Int}\;\cup\;(\textit{Val} \times \textit{Val})\;\cup\; (\textit{Val} + \textit{Val})\;\cup\;(\textit{Expr}_V \times Env) \\
 &   &\textit{Unit}&=&\{ \pmb{(\,)} \} \\
b&\in&\textit{Bool}&=& \{\textbf{False},\textbf{True}\} \\
n&\in&\textit{Int}&=& \{\ldots,-2,-1,0,1,2,\ldots\} \\
\end{array}$

$\displaystyle\begin{array}{rcl}
\textit{Expr}_V &\subset& Expr\\
\textit{Expr}_V &=& \{ \lambda x:\tau.e \mid x\in\textit{Nm}, \tau\in\textit{Ty}, e\in\textit{Expr}\}~\cup~\\
            & &\{ \textbf{rec}~f~\lambda x:\tau.e \mid f\in\textit{Nm}, x\in\textit{Nm}, \tau\in\textit{Ty}, e\in\textit{Expr}\}
\end{array}$

$~$
$~$

$\displaystyle\Downarrow ~:~ \textit{Expr}\times \textit{Env} \to \textit{Val} $

$~$

$\displaystyle\frac{}{(x, \sigma) \Downarrow \sigma(x)}$

$\displaystyle\frac{}{(\lambda x:\tau.e, \sigma) \Downarrow
                \langle\lambda x:\tau.e, \sigma\rangle}
\quad
 \displaystyle\frac{}{(\textbf{rec}~f~\lambda x:\tau.e, \sigma) \Downarrow
                \langle\textbf{rec}~f~\lambda x:\tau.e, \sigma\rangle}$

$
\displaystyle\frac{}{(\pmb{(\,)}, \sigma) \Downarrow \pmb{(\,)}}
\qquad
\displaystyle\frac{}{(n, \sigma) \Downarrow n}
\qquad
\displaystyle\frac{}{(\textbf{True}, \sigma) \Downarrow \textbf{True}}
\quad
\displaystyle\frac{}{(\textbf{False}, \sigma) \Downarrow \textbf{False}}
$

$~$

$
\displaystyle\frac{(e_1,\sigma)\Downarrow v_1\quad (e_2,\sigma)\Downarrow v_2}{(\pmb{(}e_1\pmb{,}e_2\pmb{)},\, \sigma) \Downarrow (v_1,v_2)}
\quad
\displaystyle\frac{(e,\sigma)\Downarrow (v_1,v_2)}{(\textbf{fst}\;e, \sigma) \Downarrow v_1}
\quad
\displaystyle\frac{(e,\sigma)\Downarrow (v_1,v_2)}{(\textbf{snd}\;e, \sigma) \Downarrow v_2}
$

$~$

$
\displaystyle\frac{(e,\sigma)\Downarrow v}{(\textbf{inL}\;e, \sigma) \Downarrow in_L\,v}
\quad
\displaystyle\frac{(e,\sigma)\Downarrow v}{(\textbf{inR}\;e, \sigma) \Downarrow in_R\,v}
$

$~$

$
\displaystyle\frac{(e,\sigma)\Downarrow in_L\,v\quad (e_1,\{x_1\mapsto v\}\sigma)\Downarrow v_1}{
  (\textbf{case}\;e\;\textbf{of}\,\pmb{\{}\,\textbf{inL}\,\lambda x_1:\tau_1.e_1\,\pmb{;}\;~\textbf{inR}\,\lambda x_2:\tau_2.e_2\,\pmb{\}}, \sigma) \Downarrow v_1}
$

$~$

$
\displaystyle\frac{(e,\sigma)\Downarrow in_R\,v\quad (e_2,\{x_2\mapsto v\}\sigma)\Downarrow v_2}{
  (\textbf{case}\;e\;\textbf{of}\,\pmb{\{}\,\textbf{inL}\,\lambda x_1:\tau_1..e_1\,\pmb{;}\;~\textbf{inR}\,\lambda x_2:\tau_1..e_2\,\pmb{\}}, \sigma) \Downarrow v_2}
$

$~$

$\displaystyle\frac{~
   \begin{array}{l}
   (e_1,\sigma) \Downarrow \langle\lambda x:\tau.e, \sigma_1\rangle \\
   (e_2,\sigma) \Downarrow v_2 \\
   (e, \{x\mapsto v_2\}\sigma_1) \Downarrow v
   \end{array}~}{
   (e_1~e_2, \sigma) \Downarrow v}
\qquad
 \displaystyle\frac{~
   \begin{array}{l}
   (e_1,\sigma) \Downarrow \langle\textbf{rec}~f~\lambda x:\tau.e, \sigma_1\rangle \\
   (e_2,\sigma) \Downarrow v_2 \\
   (e, \{x\mapsto v_2,\;f\mapsto\langle\textbf{rec}~f~\lambda x:\tau.e,\sigma_1\rangle\}\sigma_1) \Downarrow v
   \end{array}~}{
   (e_1~e_2, \sigma) \Downarrow v}
$

$~$

$\displaystyle\frac{
   ((\lambda x:\tau.e)~e_2, \sigma) \Downarrow v}{
   (\textbf{let}~x:\tau=e_2~e, \sigma) \Downarrow v}$

$~$


$\displaystyle\frac{~
   (e,\sigma) \Downarrow \textbf{True} \quad
   (e_1,\sigma) \Downarrow v}{
   (\textbf{if}~e~\textbf{then}~e_1~\textbf{else}~e_2, \sigma) \Downarrow v}
\qquad
 \displaystyle\frac{~
   (e,\sigma) \Downarrow \textbf{False} \quad
   (e_2,\sigma) \Downarrow v}{
   (\textbf{if}~e~\textbf{then}~e_1~\textbf{else}~e_2, \sigma) \Downarrow v}
$

$~$

$\displaystyle\frac{~
   \begin{array}{l}
   (e_1,\sigma) \Downarrow n_1\\
   (e_2,\sigma) \Downarrow n_2\\
   n = n_1 \stackrel{\textit{Int}}{+} n_2
   \end{array}~}{
   (e_1 + e_2, \sigma) \Downarrow n}
\qquad\cdots\quad\cdots
\qquad
 \displaystyle\frac{~
   \begin{array}{l}
   (e_1,\sigma) \Downarrow b_1\\
   (e_2,\sigma) \Downarrow b_2\\
   b = b_1 \land b_2
   \end{array}~}{
   (b_1 \mathop{\mathsf{\&\!\!\&}\,} b_2, \sigma) \Downarrow b}
\qquad\cdots\quad\cdots
$

$~$

$
 \displaystyle\frac{~
   \begin{array}{l}
   (e_1,\sigma) \Downarrow v_1\\
   (e_2,\sigma) \Downarrow v_2\\
   v_1 \stackrel{\tau}{=} v_2
   \end{array}~}{
   (e_1 = e_2, \sigma) \Downarrow \textbf{True}}~~(\tau\in\{\textit{Int},\textit{Bool}\})
\quad
 \displaystyle\frac{~
   \begin{array}{l}
   (e_1,\sigma) \Downarrow v_1\\
   (e_2,\sigma) \Downarrow v_2\\
   v_1 \stackrel{\tau}{\neq} v_2
   \end{array}~}{
   (e_1 = e_2, \sigma) \Downarrow \textbf{False}}~~(\tau\in\{\textit{Int},\textit{Bool}\})
\qquad\cdots\quad\cdots
$

위의 표기법은 비유하자면 $v = f(x)$라고 쓰는 대신에 $f(x)$를 기계적인 규칙을 따라 계속 계산을 돌리다 보면 $v$라는 값에 도달한다는 의미로 $f(x) \Downarrow v$라는 식으로 표시하는 것이다.
산술연산은 Int 타입의 정수값에 대해서만, 논리연산은 Bool 타입의 진리값에 대해서만, 비교연산은 Int 타입의 정수값끼리 또는 Bool 타입의 진리값끼리만 정의되어 있다. 여기서는 논리연산의 short circuit에 대해서는 일단 생각하지 않기로 하자.

In [9]:
-- finite mapping을 순서쌍 리스트 타입으로 정의
type Env = [(Nm, Val)]
-- Expr_V를 따로 정의하지 않고 그냥 Expr을 이용해 정의
data Val = VU | VI Int | VB Bool | VPair Val Val | VinL Val | VinR Val | Cl Expr Env  deriving Show

-- simga(x)에 해당하는 것이 lookup x simga
lookup' x env  = v  where Just v = lookup x env

In [10]:
import Data.List (intersperse)
dpEnv env = "{"++ concat (intersperse ", " [x++" ↦ "++dpVal v | (x,v)<-env]) ++ "}"

dpVal VU = show "()"
dpVal (VI n) = show n
dpVal (VB b) = show b
dpVal (VPair v1 v2) = "("++dpVal v1++", "++dpVal v2++")"
dpVal (VinL v) = "inL("++dpVal v++")"
dpVal (VinR v) = "inR("++dpVal v++")"
dpVal (Cl t env) = "⟨"++dpExpr t++", "++dpEnv env++"⟩"

dispEnv env = Display[html $ "<code>"++dpEnv env++"</code>"]
dispVal v = Display[html $ "<code>"++dpVal v++"</code>"]

In [32]:
-- TODO add more things
eval :: Expr -> Env -> Val
eval Unit _ = VU
eval (I n) _ = VI n
eval (B b) _ = VB b
eval v@(Lam (x,_) e)         env = Cl v env
eval v@(Rec f (Lam (x,_) e)) env = Cl v env
eval (Pair e1 e2) env = VPair v1 v2
                        where
                          v1 = eval e1 env
                          v2 = eval e2 env
eval (Fst e) env = v1
                   where VPair v1 v2 = eval e env
eval (Snd e) env = v2
                   where VPair v1 v2 = eval e env
eval (InL e) env = VinL v
                   where v = eval e env
eval (InR e) env = VinR v
                   where v = eval e env
eval (Case e (Lam (x1,_) e1) (Lam (x2,_) e2)) env =
  case eval e env of
    VinL v -> eval e1 ((x1,v):env)
    VinR v -> eval e2 ((x2,v):env)        
eval (Var x)     env = lookup' x env
eval (App e1 e2) env =
  let v2 = eval e2 env in
  case eval e1 env of
   v1@(Cl (Lam (x,_) e)         env1) -> eval e ((x,v2):env1)
   v1@(Cl (Rec f (Lam (x,_) e)) env1) -> undefined
eval (If e e1 e2) env = undefined
eval (Let (x,_) e2 e) env = undefined -- let x:t=e2 in e 는 (\x:t.e) e2 의 문법설탕(syntactic sugar)
-- 산술연산
eval (Add e1 e2)  env = VI (x+y)
  where
  VI x = eval e1 env
  VI y = eval e2 env
eval (Mul e1 e2)  env = undefined
eval (Neg e)      env = undefined
-- 논리연산
eval (And e1 e2)  env = undefined
eval (Or e1 e2)   env = undefined
eval (Not e)      env = VB (not v)
                        where VB v = eval e env
-- 비교연산
eval (Eq e1 e2)   env = undefined
eval (Ne e1 e2)   env = undefined
eval (Lt e1 e2)   env = undefined
eval (Gt e1 e2)   env = undefined
eval (Le e1 e2)   env = undefined
eval (Ge e1 e2)   env = undefined

In [12]:
-- eval을 완성한 후 factExpr를 완성하여 7팩토리알을 실행해 보라
factExpr = Rec undefined undefined
eval (Let "fact" factExpr $ Var "fact" `App` I 7) []

<interactive>:1:11: error:
    • Couldn't match expected type ‘(Nm, Ty)’ with actual type ‘String’
    • In the first argument of ‘Let’, namely ‘"fact"’
      In the expression: Let "fact" factExpr
      In the first argument of ‘eval’, namely ‘(Let "fact" factExpr $ Var "fact" `App` I 7)’

In [13]:
:type eval
:type uncurry eval

eval (App (Lam ("x",INT:->INT) (Var "x")) (Lam ("x",INT) (Var "x"))) []

sigma = [("twice",Cl (Lam ("f",INT:->INT) (Lam ("x",INT) (App f(App f x)))) [])]
      where
       f = Var "f"
       x = Var "x"

twice = Var "twice"

putStr . dpVal $ eval (App twice twice) sigma

eval :: Expr -> Env -> Val

uncurry eval :: (Expr, Env) -> Val

Cl (Lam ("x",INT) (Var "x")) []

⟨λx:int.f (f x), {f ↦ ⟨λf:int->int.λx:int.f (f x), {}⟩}⟩

In [13]:
dispEnv sigma

{twice ↦ ⟨λf:int->int.λx:int.f (f x), {}⟩}

In [14]:
putStrLn . dpVal $ eval (App twice twice) sigma
dispVal $ eval (App twice twice) sigma

⟨λx:int.f (f x), {f ↦ ⟨λf:int->int.λx:int.f (f x), {}⟩}⟩

⟨λx:int.f (f x), {f ↦ ⟨λf:int->int.λx:int.f (f x), {}⟩}⟩

In [15]:
idExpr
putStr $ dpExpr idExpr

Lam ("x",INT) (Var "x")

λx:int.x

In [16]:
putStrLn . dpVal $ eval (I 3) sigma
dispVal $ eval (I 3) sigma

doubleExpr = Lam ("x",INT) (Var "x" `Add` Var "x")

dispVal $ eval (App idExpr (I 3)) sigma
dispVal $ eval (App doubleExpr (I 3)) sigma

3

3

3

6

In [17]:
putStrLn . dpVal $ eval (B True) sigma
dispVal $ eval (B True) sigma

True

True

In [25]:
eval (Pair (I 3) (B True)) []
putStrLn . dpVal $ eval (Pair (I 3) (B True)) []

VPair (VI 3) (VB True)

(3, True)

In [26]:
pairTm = (Pair (I 3) (B True))
putStrLn . dpVal $ eval (Fst pairTm) []
putStrLn . dpVal $ eval (Snd pairTm) []

Line 1: Redundant bracket
Found:
(Pair (I 3) (B True))
Why not:
Pair (I 3) (B True)

3

True

In [34]:
caseTm = -- int + bool 타입을 처리하는 case 식
  Case (InL (I 3))
  -- Case (InR (B False)) 
    (Lam ("x1", INT) $ Add (Var "x1") (I 3)) -- inL 처리
    (Lam ("x2", INT) $ Not (Var "x2")) -- inR 처리
eval caseTm []

VI 6